# Simulations for evaluating archaic ancestry inference

## Set paths and import libraries

In [1]:
# run in archanc root directory
import os
proj_dir = os.getcwd()
msprime_dir = proj_dir +"/output/msprime/"
archie_src_dir = proj_dir + "/src/ArchIE/"
archie_out_dir = proj_dir + "/output/ArchIE/"

In [5]:
import sys
sys.path.insert(0, proj_dir + '/src/stdpopsim')

from stdpopsim import homo_sapiens, models
import msprime
import itertools
import random
import numpy as np
import pandas as pd
import math
import cyvcf2

## Classes for generating EIGENSTRAT-formatted output from the `msprime` tree sequence

### Simulating MNMs
The function `generateEigData.mnms()` will randomly select a fraction of variant sites to be MNMs. The second mutation in each MNM is randomly placed up to K bp downstream of the first, and the genotypes of the first mutation are duplicated. 

In [9]:
class mnmEigData:
    """
    object defines .geno and .snp matrices 
    """

    def __init__(self, geno, snp, prefix, repeat_rows): 
        self.geno = geno
        self.snp = snp
        self.prefix = prefix
        self.repeat_rows = repeat_rows


class generateEigData:
    """
    class describes the .snp, .geno, and .ind EIGENSTRAT formats
    """
    
    def __init__(self, ts, model_label, rep_label=0, mnm_frac=0, mnm_dist=0):
        self.ts = ts
        self.model_label = model_label
        self.rep_label = rep_label
        if mnm_frac > 0:
            self.sim_mnms = True
        else:
            self.sim_mnms = False
        self.mnm_frac = mnm_frac
        self.mnm_dist = mnm_dist
        self.geno = self.geno()
        self.snp = self.snp()
        self.prefix = self.prefix()
    
    def prefix(self):
        """
        get unique identifier for model based on inputs
        """

        prefix = self.model_label + "_rep" + str(self.rep_label)
#         if self.sim_mnms:
#             prefix = prefix + "_mnm"+str(self.mnm_dist)+"-"+str(self.mnm_frac)
        
        return prefix
    
    def geno(self):
        """
        generate .geno file from tree sequence
        """

        geno = np.zeros(200, dtype=np.int8)
        
        for variant in list(self.ts.variants()):
            geno = np.vstack([geno, variant.genotypes])
        
        geno = np.delete(geno, (0), axis=0) # remove dummy 1st row
        return geno
                
    def snp(self):
        """
        generate .snp file from tree sequence
        """

        d = pd.DataFrame(columns=['ID', 'CHR', 'POS', 'POS1', 'REF', 'ALT'])
        for variant in list(self.ts.variants()):
            d = d.append(pd.DataFrame({'ID': "1:"+str(round(variant.site.position)),
                      'CHR': "1",
                      'POS': str(round(variant.site.position)),
                      'POS1': str(variant.site.position/10e6), 
                      'REF': "A", 
                      'ALT': "G"
                     }, index=[0]), ignore_index=True)

        snp_df = pd.DataFrame(d)
        return snp_df
    
    def mnms(self):
        """
        update .geno and .snp output to include simulated MNMs
        """
        
        snp_mnm = pd.DataFrame(columns=['ID', 'CHR', 'POS', 'POS1', 'REF', 'ALT'])
#         geno_mat = self.geno()
        repeat_rows = []
        for index, snp in self.snp.iterrows():
            snp_mnm = snp_mnm.append(snp, ignore_index=True)
            
            random.seed(snp['POS'])
            if random.random() < self.mnm_frac:

                dist = random.randint(1, self.mnm_dist)

                mnm_snp = snp
                mnm_snp['POS'] = str(round(int(snp['POS'])+dist))
                mnm_snp['POS1'] = float(mnm_snp['POS'])/10e6
                mnm_snp['ID'] = "1:" + mnm_snp['POS']
                snp_mnm = snp_mnm.append(mnm_snp, ignore_index=True)
                repeat_rows.append(2)
            else:
                repeat_rows.append(1)
        
        geno_mat_mnm = np.repeat(self.geno, repeats=repeat_rows, axis=0)

        mnm_prefix = self.prefix + "_mnm"+str(self.mnm_dist)+"-"+str(self.mnm_frac)
        
        return mnmEigData(geno=geno_mat_mnm,
                          snp=snp_mnm,
                          prefix=mnm_prefix,
                          repeat_rows=repeat_rows)

    
class writeEigData:
    """
    functions for writing .snp .geno and .ind files
    """
    
    def __init__(self, eig_data, output_dir="./"):
        self.eig_data = eig_data
        self.prefix = eig_data.prefix
        self.output_dir = output_dir

    def dump(self):
        """
        run all 3 output functions: write_snp, write_geno, and write_ind
        """
        
        self.write_snp()
        self.write_geno()
        self.write_ind()
        
    def write_snp(self):
        """
        write .snp files
        """

        self.eig_data.snp.to_csv(self.output_dir + self.prefix + ".snp", 
                  index=True,
                  sep="\t")

    def write_geno(self):
        """
        write .geno files
        """

        geno_afr = self.eig_data.geno[:,:100]
        geno_eur = self.eig_data.geno[:,100:200]

        geno_pop = [geno_afr, geno_eur]

        for i, pop in enumerate(["afr", "eur"]):
            np.savetxt(self.output_dir + self.prefix + "_" + pop + ".geno", geno_pop[i], delimiter="", fmt='%i')

    def write_ind(self):
        """
        write .ind files
        """

        for pop in ["afr", "eur"]:
            # write out separate .ind files per population.
            # columns indicate sample ID, sex (set as 'U'), and label (set as 'ADMIXED')
            ind_file = self.output_dir + self.prefix + "_" + pop + ".ind"
            with open(ind_file, "w") as id_file:
                for sample_id in range(0,100):
                    sample_name = self.prefix + "_" + pop + \
                        "_sample_" + str(sample_id)
                    print("\t".join([sample_name, "U", pop]),  file=id_file)

In [31]:
class mnmVCFData:
    """
    object defines .vcf matrices
    """

    def __init__(self, vcf, prefix, repeat_rows):
        self.vcf = vcf
        self.prefix = prefix
        self.repeat_rows = repeat_rows

class generateVCFData:
    """
    generate VCF files from tree sequence
    """

    def __init__(self, ts, model_label, rep_label=0, mnm_frac=0, mnm_dist=0):
        self.ts = ts
        self.model_label = model_label
        self.rep_label = rep_label
        if mnm_frac > 0:
            self.sim_mnms = True
        else:
            self.sim_mnms = False
        self.mnm_frac = mnm_frac
        self.mnm_dist = mnm_dist
        self.vcf = self.vcf()
        self.prefix = self.prefix()

    def prefix(self):
        """
        get unique identifier for model based on inputs
        """

        prefix = self.model_label + "_rep" + str(self.rep_label)
        return prefix


    def vcf(self):
        """
        generate data frame of genotypes in VCF format
        """

        # for variant in self.ts.variants():
        #     pos = round(variant.site.position)
            # output current variant to VCF

                # text_file.write("\t".join(snv) + "\n")

        snvs = []
        # d = np.empty([59, 2], dtype=int)
        for variant in list(self.ts.variants()):
            pos = round(variant.site.position)
            snv = ["1", str(pos), "1_%s" % pos, "A", "G", "50", "PASS", "VT=SNP", "GT"]
            snv.extend([("|").join([str(g) for g in variant.genotypes[i: i+2]]) for i in range(0, len(variant.genotypes), 2)])
            snvs.append(snv)
            
#         d = d.append(pd.DataFrame(snv, index=[0]), ignore_index=True)
        snv_df = pd.DataFrame(snvs)
        return snv_df
    
    def mnms(self):
        """
        update .vcf output to include simulated MNMs
        """

        snv_df = pd.DataFrame()
#         geno_mat = self.geno()
        repeat_rows = []
        for index, snv in self.vcf.iterrows():
            snv_df.append(snv, ignore_index=True)

            random.seed(int(snv[1]))
            if random.random() < self.mnm_frac:
                # make sure a mnm is at least 10bp from its counterpart.
                # this is to avoid the internal filter of Sprime.
                mnm_snv = snv
                dist = random.randint(10, self.mnm_dist)
                mnm_cand = int(snv[1]) + dist
                mnm_cand_r = str(round(mnm_cand))
                mnm_snv[1] = str(mnm_cand_r)
                mnm_snv[2] = "1_%s" % mnm_cand_r
                snv_df.append(snv_df, ignore_index=True)
                repeat_rows.append(2)
            else:
                repeat_rows.append(1)

        mnm_prefix = self.prefix + "_mnm"+str(self.mnm_dist)+"-"+str(self.mnm_frac)

        return mnmVCFData(vcf=snv_df,
                          prefix=mnm_prefix,
                          repeat_rows=repeat_rows)

## Simulate models

Simulate 200 samples (100 each of European and African ancestry) under each of the specified models (with 1000 replicates each)

In [6]:
# coalescent simulation parameters
sample_size = 100 # each
length = 50000
mu = 1.15e-8
rr = 1e-8
replicates = 1000
seed = 30

# Gutenkunst 3-population model
GutenkunstThreePop_model = homo_sapiens.GutenkunstThreePopOutOfAfrica()
GutenkunstThreePop_ts = msprime.simulate(
    # first 100 samples from AFR, next 100 from EUR
    samples=[msprime.Sample(0, 0)]*sample_size + [msprime.Sample(1, 0)]*sample_size,
    length=length, 
    mutation_rate=mu, 
    recombination_rate=rr,
    random_seed=seed,
    num_replicates=replicates,
    **GutenkunstThreePop_model.asdict())

# Tennessen 2-population model
TennessenTwoPop_model = homo_sapiens.TennessenTwoPopOutOfAfrica()
TennessenTwoPop_ts = msprime.simulate(
    # first 100 samples from AFR, next 100 from EUR
    samples=[msprime.Sample(0, 0)]*sample_size + [msprime.Sample(1, 0)]*sample_size,
    length=length, 
    mutation_rate=mu, 
    recombination_rate=rr,
    random_seed=seed,
    num_replicates=replicates,
    **TennessenTwoPop_model.asdict())

#-------------------------------------------------------
# define other models here and add to model_dict below
#-------------------------------------------------------

# modify demographic parameters to include archaic branches
# GutenkunstThreePopArchaic_model = homo_sapiens.GutenkunstThreePopArchaic()

# GutenkunstThreePopArchaic_ts = msprime.simulate(
#     # first 100 samples from AFR, next 100 from EUR
#     samples=[msprime.Sample(0, 0)]*sample_size + [msprime.Sample(1, 0)]*sample_size,
#     length=length, 
#     mutation_rate=mu, 
#     recombination_rate=rr,
#     random_seed=seed,
#     num_replicates=replicates,
#     **GutenkunstThreePopArchaic_model.asdict())

# create dictionary of models
model_dict = {"GutenkunstThreePop": GutenkunstThreePop_ts,
             "TennessenTwoPop": TennessenTwoPop_ts}

In [37]:
TennessenTwoPop_ts = msprime.simulate(
    # first 100 samples from AFR, next 100 from EUR
    samples=[msprime.Sample(0, 0)]*sample_size + [msprime.Sample(1, 0)]*sample_size,
    length=length, 
    mutation_rate=mu, 
    recombination_rate=rr,
    random_seed=seed,
    num_replicates=replicates,
    **TennessenTwoPop_model.asdict())

In [7]:
from stdpopsim import homo_sapiens, models
import msprime
TennessenTwoPop_model = homo_sapiens.TennessenTwoPopOutOfAfrica()
TennessenTwoPop_model.debug()

Epoch: 0 -- 204.6 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |4.32e+05 1.45e+04         0.0166 |     0        0    
1 |5.01e+05 9.28e+03         0.0195 |     0        0    

Events @ generation 204.6
   - Migration rate change for (0, 1) to 2.5e-05
   - Migration rate change for (1, 0) to 2.5e-05
   - Population parameter change for 1: initial_size -> 9279.212349452768 growth_rate -> 0.00307 
   - Population parameter change for 0: initial_size -> 432124.58438330283 growth_rate -> 0 
Epoch: 204.6 -- 920.0 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |4.32e+05 4.32e+05              0 |     0     2.5e-05 
1 |9.28e+03 1.03e+03        0.00307 |  2.5e-05     0    

Events @ generation 920.0
   - Migration rate change for (0, 1) to 0.00015
   - Migration rate change for (1, 0) to 0.00015
   - Population parameter change for 1:

# Get simulated data and run archaic admixture detection methods

In [34]:
run_archie = False
for model_label, model in model_dict.items():    
    for j, ts in enumerate(model):
        if j == 1: # testing with just 1st simulated tree sequence
            
#             eig_data = generateEigData(ts, model_label, rep_label=j, mnm_frac=0.015, mnm_dist=100)
            eig_data = generateVCFData(ts, model_label, rep_label=j, mnm_frac=0.015, mnm_dist=100)
            print("---Without MNMs---")
            print(eig_data.prefix)
#             print(eig_data.geno.shape)
#             print(eig_data.snp.shape)
            print(eig_data.vcf)
#             writeEigData(eig_data, msprime_dir).dump()
            
            # write non-MNM data to VCF
#             ts.write_vcf(msprime_dir + eig_data.prefix + ".vcf", 2)
            
            print("---With MNMs---")
            eig_data_mnms = eig_data.mnms()
            print(eig_data_mnms.prefix)
#             print(eig_data_mnms.geno.shape)
#             print(eig_data_mnms.snp.shape)
            print(eig_data.vcf)
#             writeEigData(eig_data_mnms, msprime_dir).dump()
            
            # add code for converting .snp/.geno/.ind data to VCF
            # alternatively, use cyvcf2 to read the non-MNM VCF into a numpy array 
            # and duplicate rows with the repeat_rows indices
            #
            # old_gts = VCF genotypes as numpy array
            # ...
            # new_gts = np.repeat(self.geno, repeats=eig_data_mnms.repeat_rows, axis=0)
            # ...
            # write new VCF
            
            if run_archie:
                for data in [eig_data, eig_data_mnms]:
                    for pop in ["afr", "eur"]:

                        if pop == "afr":
                            ref_pop = "eur"
                        else:
                            ref_pop = "afr"

                        prefix = data.prefix
                        stats_pop_cmd = "python " + archie_src_dir + "data/calc_stats_window_data.py" + \
                            " -s " + msprime_dir + prefix + ".snp" + \
                            " -i " + msprime_dir + prefix + "_" + pop + ".ind" + \
                            " -a " + msprime_dir + prefix + "_" + pop + ".geno" + \
                            " -r " + msprime_dir + prefix + "_" + ref_pop + ".geno" + \
                            " -c 1 -b 0 -e 50000 -w 50000 -z 50000 " + \
                            " > " + archie_out_dir + prefix + "_" + pop + ".txt"  
                        print(stats_pop_cmd + "\n")
#                         os.system(stats_pop_cmd)
            
            # add code for evaluating additional methods
            # if run_sprime:
            # ...
            # ...
            
            # if run_moments:
            # ...
            # ...
            
            # if run_idetect:
            # ...
            # ...

---Without MNMs---
GutenkunstThreePop_rep1
    0      1        2   3   4   5     6       7   8    9   ...   99   100  \
0     1     27     1_27   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
1     1     75     1_75   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
2     1    516    1_516   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
3     1    753    1_753   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
4     1   1543   1_1543   A   G  50  PASS  VT=SNP  GT  1|0 ...   0|1  1|0   
5     1   1697   1_1697   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
6     1   2001   1_2001   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
7     1   2427   1_2427   A   G  50  PASS  VT=SNP  GT  1|1 ...   1|1  1|1   
8     1   2718   1_2718   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
9     1   2873   1_2873   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
10    1   2944   1_2944   A   G  50  PASS  VT=SNP  GT  0|0 ...   0|0  0|0   
11    1   3086   1_3086   A   G  